In [ ]:
# From Objects to Databases: Receipt Tracking System

# From Objects to Databases: Receipt Tracking System

This module covers databases, which are critical for storing and working with data in real-world applications. The approach and frameworks will use here to work with databases also reinforces Object-Oriented Programming concepts covered so fa, as the same principles of classes, attributes, methods, and inheritance apply directly to database design.

This module will not work with SQL (Structured Query Language). Instead, will use use an ORM (Object-Relational Mapping) tool. An ORM is a programming technique that maps Python classes to database tables, allowing you to interact with databases using Python code rather than SQL queries. With an ORM, a Python class definition becomes a table schema, class instances become database rows, and Python methods perform database operations—all without writing SQL statements.

This module covered into SQLAlchemy, the most widely used ORM in Python.

In [ ]:

## Lesson 1: From Python Objects to Database Tables


Without persistent storage, every receipt that is parsed will be lost when the program terminates. Data only exists in memory during program execution and disappears when the program ends.


Why not JSON files? JSON is the most commonly used file format for data interchange, and most data these days is delivered as JSON files. However, saving data as JSON makes it difficult to use in applications. For each query, the entire file must be read into memory and custom logic must be written to find the desired information.

Consider these example queries:

    * Find all Starbucks receipts from January 1 to March 15 where the total is between $12 and less than $100 and that have at least 2 items
    * Find all receipts where a transaction amount is more than $500

The logic for each of these queries needs to be implemented independently. Additionally, JSON files must be parsed one record at a time, making operations slow on large datasets. Databases store data in a structured format that makes it easy to formulate and execute complex queries efficiently.


Databases can be local (installed on the local machine) or cloud-based (hosted remotely and managed by a service provider). Local databases vary in sophistication, with different products offering different features. Cloud databases can be accessed from anywhere with an internet connection, similar to the difference between using Excel locally versus using Google Sheets in the cloud.

While database engineers often access and manipulate databases manually through database management tools, building applications requires programmatic access. This course relies primarily on using SQLAlchemy ORM for database interaction. SQLAlchemy is installed using `pip`, the Python package manager, with the command `pip install sqlalchemy`, following the same process used for other Python packages. SQLAlchemy makes each Python class definition correspond to a database table. The concept of a table is similar to a spreadsheet. A database table contains related information grouped together. For example, an employee table might contain `first_name`, `last_name`, `age`, `department`, `address`, and `salary`. The concept of encapsulation applies here in the same way we covered in the OOP module. Another related table might contain employee paychecks, storing `paycheck_date`, `amount`, `deductions`, and `employee_ID` linking back to the employee table. This structure mirrors how information is encapsulated into classes in object-oriented programming. When data is grouped together logically in a Python class, that same class can be used as a database table definition.

The course covers basic operations to Create, Retrieve, Update, and Delete (CRUD) data without writing SQL.

In [ ]:
# What they already know - storing receipts in objects
class Receipt:
    def __init__(self, merchant, phone, total, tax, grand_total):
        self.merchant = merchant
        self.phone = phone
        self.total = total
        self.tax = tax
        self.grand_total = grand_total

# Create some receipts
receipts = [
    Receipt("Starbucks", "555-0100", 12.50, 1.13, 13.63),
    Receipt("Target", "555-0200", 45.99, 3.68, 49.67),
]

# Calculate total spending
total = sum(r.grand_total for r in receipts)
print(f"Total spent: ${total:.2f}")


In [ ]:
Suppose you exit a person into the AI, a bomb in the AI. What happens if you restart your kernel? You will restart information, and you have to re-execute the AI to generate the receipts because you received. 
Natually, if you restart your kernel, you will look

In [ ]:
### Database Solutions for Persistent Data Storage

Applications require several key capabilities for effective data management. Data must survive when the program closes, a property known as persistence. The system must support searching and filtering through queries. Analyzing trends over time requires aggregation functions, such as averaging all receipt totals by month or calculating the median item value. Ideally, these operations should be possible without writing a specialized query language, though the ability to use SQL when needed remains valuable. Finally, data access must extend beyond the local machine, enabling queries from web applications or other remote environments.
SQLAlchemy ORM addresses these requirements. ORM stands for Object-Relational Mapping, a technique where Python class definitions correspond directly to database tables. This approach eliminates the need to write SQL strings, allowing developers to work entirely in Python.
Consider the difference in approaches. Without an ORM, adding a new receipt where the merchant is Starbucks and the total is $12.50 requires combining SQL within Python code:

```python
# Without ORM (what we WON'T do):

cursor.execute("INSERT INTO receipts (merchant, total) VALUES (?, ?)", 
               ("Starbucks", 12.50))
With an ORM, the same operation uses natural Python syntax that remains clear even to those unfamiliar with the language:
```
```python# With ORM (what we WILL do):
receipt = Receipt(merchant="Starbucks", total=12.50)
session.add(receipt)
session.commit()
```

The ORM approach treats database operations as object (instance) manipulations, making the code more intuitive and maintainable.

In [ ]:
    
### Installation & Setup (10 minutes)

!pip install sqlalchemy


Setting up database interaction requires importing the correct modules and classes and creating a connection to the database. For simplicity, this course uses `SQLite`, a widely used database management system for prototyping and testing. `SQLite` creates the database in a single file that contains everything needed to store and interact with data.

The basic setup code establishes the database connection and creates the necessary infrastructure:

```python
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create a database engine (SQLite = local file)
engine = create_engine('sqlite:///receipts.db', echo=True)

# Base class for all our database models
Base = declarative_base()

# Session factory - this is how we talk to the database
Session = sessionmaker(bind=engine)
session = Session()
```

This code introduces several key concepts. The engine represents the connection to the database file. The Base class serves as the parent class for all database-backed classes -- remember a databse base class is the Python equivalent of of the table we will be interacting with. This establishes the foundation for the ORM mapping. The session acts as a workspace for making changes to the database, similar to a transaction in which operations can be grouped together before being committed. The `echo=True` parameter causes SQLAlchemy to display the SQL statements it generates, which can be valuable for learning SQL and debugging if you're already somewhat familir with `SQL` and want to see the commands being executed.

### Creating Database Tables from Python Classes

A simple example demonstrates how the system works. Consider creating a receipt table to store receipt data. Recall that a table is a collection of related data, similar to a spreadsheet tab. The data is related in the same way objects are related in OOP. Below is how a receipt table is created using SQLAlchemy:

```python
from datetime import date

class Receipt(Base):
    __tablename__ = 'receipts'
    
    # Define columns - just like class attributes!
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def __repr__(self):
        return f"<Receipt(merchant='{self.merchant}', total=${self.total:.2f})>"

# Create all tables
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()
```

This code performs several operations. The `__tablename__` attribute specifies the name of the table in the database. Each `Column()` definition creates a database field, functioning like class attributes but with database-specific properties (this is similar to columns in the spreadsheet analogy). Column types map Python data types to database types: `Integer` for whole numbers (like the id field), `String` for text (like merchant), `Float` for decimal numbers (like total), and `Date` for date values. The `primary_key=True` parameter designates the id field as a unique identifier that auto-increments with each new record. The `nullable=False` parameter marks a field as required, preventing `None` values. The `default=date.today` parameter provides an automatic value when none is specified. Finally, `Base.metadata.create_all(engine)` executes the actual table creation in the database, translating the Python class definition into the corresponding database schema.


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.orm import sessionmaker

# Create a database engine (SQLite = local file)
engine = create_engine('sqlite:///receipts.db', echo=True)

# Base class for all our database models
Base = declarative_base()

# Session factory - this is how we talk to the database
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from datetime import date

class Receipt(Base):
    __tablename__ = 'receipts'
    
    # Define columns - just like class attributes!
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def __repr__(self):
        return f"<Receipt(merchant='{self.merchant}', total=${self.total:.2f})>"

# Create all tables
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()


### CRUD Operations 

Creating new database records involves instantiating Python objects and adding them to the session:



In [ ]:
from datetime import date

# Create receipt objects (just like any Python class!)
receipt1 = Receipt(
    merchant="Starbucks",
    total=12.50,
    date=date(2025, 10, 15)
)

receipt2 = Receipt(
    merchant="Target",
    total=45.99,
    date=date(2025, 10, 16)
)

receipt3 = Receipt(
    merchant="Whole Foods",
    total=67.50
    # date will default to today since we didn't specify it
)

# Add to the session (staging area)
session.add(receipt1)
session.add(receipt2)
session.add(receipt3)

# Save to database
session.commit()


In [ ]:
The `session` acts as a staging area. When `session.add()` is called, the object is tracked by the session but **not yet written to the database**. The data only exists in Python's memory at this point. The `commit()` method executes the actual database INSERT statements, persisting all pending changes to the database file. Only after `commit()` completes does the data become permanent. `SQLAlchemy` then automatically retrieves and assigns the auto-incremented ID value to each object, which become part of the instances we created


In [ ]:

print(f"Saved receipt with ID: {receipt3.id}")  # ID is auto-assigned after the commit


**READ - Getting receipts back:**

To demonstrate that data truly persists in the database (not just in RAM), restart the Jupyter notebook kernel now. This clears all variables from memory, simulating closing and reopening the program. After restarting, run the setup code again to reconnect to the database. Here, we need to redefine the Receipt Class to provide Python with the structure of the object we will be retrieving, but in a real app, all SQLAlchemy base classes should be defined in an external file and important as needed.



In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.orm import declarative_base, sessionmaker
from datetime import date

# Reconnect to the same database file
engine = create_engine('sqlite:///receipts.db', echo=True)
Base = declarative_base()

class Receipt(Base):
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def __repr__(self):
        return f"<Receipt(merchant='{self.merchant}', total=${self.total:.2f})>"

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
all_receipts = session.query(Receipt).all()

In [ ]:
len(all_receipts)

In [ ]:
# We get a strig representation form the `__repr__`
for r in all_receipts:
    print(r)


In [ ]:
receipt = session.get(Receipt, 1)
print(f"\nReceipt #1: {receipt}")

In [ ]:
receipt = session.get(Receipt, 3)
print(f"\nReceipt #3: {receipt}")

In [ ]:
receipt = session.get(Receipt, 4)
print(f"\nReceipt #4 is: {receipt}")

In [ ]:
starbucks = session.query(Receipt).filter_by(merchant="Starbucks").all()

print("\n=== Starbucks Receipts ===")
for r in starbucks:
    print(f"  {r}")

In [ ]:

# Filter with conditions
expensive = session.query(Receipt).filter(Receipt.total > 50).all()

print("\n=== Receipts Over $50 ===")
for r in expensive:
    print(f"  {r}")


In [ ]:
# Filter by date range
from datetime import date, timedelta
week_ago = date.today() - timedelta(days=7)
recent = session.query(Receipt).filter(Receipt.date >= week_ago).all()

print("\n=== Receipts from Last Week ===")
for r in recent:
    print(f"  {r}")

Note that the the receipts still exist because they were written to the `receipts.db` file. Restarting the kernel cleared Python's memory, but the database file persisted on disk. This demonstrates the fundamental purpose of databases: data survives beyond the program's execution. In fact , you could share the file with colleagues and they would be able connect to and query it the same way we did.

The `query()` method returns a query object that can be refined with filters. The `all()` method executes the query and returns a list of results. The `get()` method retrieves a single record by its primary key. The `filter_by()` method accepts keyword arguments for simple equality comparisons, while `filter()` accepts more complex boolean expressions.


In [ ]:
**UPDATE - Changing receipts:**

Modifying existing records involves retrieving the object, changing its attributes, and committing the changes:


In [ ]:
receipt = session.get(Receipt, 1)
print(f"Before: {receipt}\n\n")

# Change it (just like a regular object!)
receipt.total = 15.00
receipt.merchant = "Starbucks Coffee"

# Save changes
session.commit()
print(f"\n\nAfter: {receipt}")

SQLAlchemy tracks changes to objects that belong to a session. When `commit()` is called, it automatically generates and executes the appropriate UPDATE statements.


In [ ]:
**DELETE - Removing receipts:**

Deleting records uses the `delete()` method:


In [ ]:
# Get a receipt
receipt = session.query(Receipt).get(1)

# Delete it
session.delete(receipt)
session.commit()
print(f"Deleted receipt from {receipt.merchant}")



The `delete()` method marks the object for deletion. The actual deletion occurs when `commit()` is called. Once deleted, the object still exists in Python memory but no longer has a corresponding database record.

### Hands-On Exercise (15 minutes)

**Exercise: replicate the abobe and build your own receipt database**

Students write this complete program:

```python
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import date

# Setup database
engine = create_engine('sqlite:///my_receipts.db')
Base = declarative_base()

class Receipt(Base):
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    phone = Column(String)
    total = Column(Float, nullable=False)
    tax = Column(Float)
    grand_total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def __repr__(self):
        return f"<Receipt({self.merchant}: ${self.grand_total:.2f})>"

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Add 5 receipts
receipts = [
    Receipt(merchant="Starbucks", phone="555-0100", total=12.50, tax=1.13, grand_total=13.63),
    Receipt(merchant="Target", phone="555-0200", total=45.99, tax=3.68, grand_total=49.67),
    Receipt(merchant="Starbucks", phone="555-0100", total=8.25, tax=0.74, grand_total=8.99),
    Receipt(merchant="Whole Foods", phone="555-0300", total=67.50, tax=5.40, grand_total=72.90),
    Receipt(merchant="Target", phone="555-0200", total=23.50, tax=1.88, grand_total=25.38),
]

for r in receipts:
    session.add(r)
session.commit()

# Query them
print("All receipts:", session.query(Receipt).count())
print("\nStarbucks receipts:")
for r in session.query(Receipt).filter_by(merchant="Starbucks").all():
    print(f"  {r}")

# Calculate total
total = sum(r.grand_total for r in session.query(Receipt).all())
print(f"\nTotal spent: ${total:.2f}")
```

### Wrap-Up
- **Key Insight:** Your Python class IS your database table!
- **No SQL needed:** It's all just Python
- **The session:** Think of it like a shopping cart - add things, then `commit()` to save
- **Preview:** Next class we'll add useful methods and better queries
- **Homework:** Add 10 of your own receipts to the database

## 2. Building a Complete Receipt Manager

The Receipt class created in the previous section focused primarily on data storage through column definitions. However, database-backed classes can also contain methods that operate on that data. Beyond the basic instance methods used in object-oriented programming, Python supports class methods, which operate on the class itself rather than individual instances -- more on this later. This section explores how to add both instance methods and class methods to enhance the Receipt class.

So far, the operations performed have been limited to basic CRUD functions: creating, reading, updating, and deleting individual records. Database systems support more sophisticated operations including sorting results by specific criteria, grouping records that share common attributes, and aggregating data through functions like sum, average, and count. SQLAlchemy provides these advanced querying capabilities without requiring SQL knowledge, maintaining the Python-based approach established in the previous section.

However, before we proceed, we will first explore how we can turn the database setup code we wrote earlier into a proper OOP class we can use to  interact with the database.

### Creating a Database Manager

The current approach to database setup involves writing repetitive code to create engines, establish connections, create tables, and manage sessions. Following object-oriented programming principles, this functionality should be encapsulated in a dedicated class that handles database operations consistently. A DatabaseManager class centralizes database configuration and provides methods for common operations like getting sessions, creating tables, and dropping tables.

```mermaid
classDiagram
    class DatabaseManager {
        -engine
        -Session
        +\_\_init\_\_(db_url)
        +get_session()
        +create_tables(Base)
        +drop_tables(Base)
    }
```

The DatabaseManager class consolidates database access logic. In a real application, this file can live in its module (say database.py) and be imported as needed.


In [4]:
# database.py - reusable database setup
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker, scoped_session

class DatabaseManager:
    def __init__(self, db_url='sqlite:///receipts.db'):
        self.engine = create_engine(db_url, echo=False)
        self.Session = scoped_session(sessionmaker(bind=self.engine))
        
    def get_session(self):
        """Get a new database session"""
        return self.Session()
    
    def create_tables(self, Base):
        """Create all tables"""
        Base.metadata.create_all(self.engine)
    
    def drop_tables(self, Base):
        """Drop all tables (careful!)"""
        Base.metadata.drop_all(self.engine)


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from datetime import date

from sqlalchemy.orm import declarative_base

Base = declarative_base()

class Receipt(Base):
    __tablename__ = 'receipts'
    
    # Define columns - just like class attributes!
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def __repr__(self):
        return f"<Receipt(merchant='{self.merchant}', total=${self.total:.2f})>"

In [ ]:

# In your main application file Global instance
db_manager = DatabaseManager()
session = db_manager.get_session()
session.query(Receipt).filter_by(merchant='Starbucks Coffee').all()

This approach provides several advantages. The database URL can be changed in one location rather than throughout the codebase. The `scoped_session` ensures thread-safe session management. Methods like `create_tables()` and `drop_tables()` provide clear, descriptive names for metadata operations. A global instance allows importing and using the manager across multiple modules without recreating connections.



### Advanced Querying Operations

SQLAlchemy supports sorting, grouping, and aggregation through its query interface, allowing complex data analysis without writing SQL statements. The previous section demonstrated how `filter()` works. The other query methods follow the same pattern, and modern language models understand exactly how to construct these queries when provided with a description of the model (class or table structure), so there is little value in covering this in more detials. The examples here serve an illustrative purpose: understanding what the code means and knowing what questions to ask when working with databases.

**Sorting Results:**

Query results can be ordered using the `order_by()` method:

```python
from sqlalchemy import desc

# Sort by date (oldest first, ascending order by default)
receipts = session.query(Receipt).order_by(Receipt.date).all()

# Sort by total (highest first -- desc for descending)
receipts = session.query(Receipt).order_by(desc(Receipt.total)).all()

# Multiple sort criteria
receipts = session.query(Receipt).order_by(
    Receipt.merchant,  # First by merchant name
    desc(Receipt.total)  # Then by total (descending)
).all()
```

**Grouping and Aggregation:**

The `func` module provides SQL aggregate functions accessible through Python:

```python
from sqlalchemy import func

# Count receipts by merchant and calculate total spending by merchant
results = session.query(
    Receipt.merchant,
    func.sum(Receipt.total).label('total')
).group_by(Receipt.merchant).all()

for merchant, total in results:
    print(f"{merchant}: ${total:.2f}")

# Average receipt amount
avg_total = session.query(func.avg(Receipt.total)).scalar()
print(f"Average receipt: ${avg_total:.2f}")

# Maximum and minimum
max_total = session.query(func.max(Receipt.total)).scalar()
min_total = session.query(func.min(Receipt.total)).scalar()
```


### Adding Instance Methods

Instance methods, as we've seen in OOP intro, operate on individual Receipt objects and can provide calculated properties or formatted output:

In [2]:

from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from datetime import date

from sqlalchemy.orm import declarative_base

Base = declarative_base()


class Receipt(Base):
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    total = Column(Float, nullable=False)
    date = Column(Date, default=date.today)
    
    def is_expensive(self):
        """Check if receipt total exceeds threshold"""
        return self.total > 100
    
    def days_since_purchase(self):
        """Calculate days between purchase and today"""
        return (date.today() - self.date).days
    
    def __repr__(self):
        return f"<Receipt(merchant='{self.merchant}', total=${self.total:.2f})>"


In [5]:
db_manager = DatabaseManager()
session = db_manager.get_session()


In [7]:
receipt = session.get(Receipt, 1)

In [9]:
if receipt.is_expensive():
    print(f"High-value purchase: {receipt.total}")
else:
    print(f"low-value purchase: {receipt.total}")


low-value purchase: 15.0


In [10]:

print(f"Purchased {receipt.days_since_purchase()} days ago")


Purchased 5 days ago


Instance methods enhance the object's behavior while keeping the code organized within the class definition. They provide a clean interface for common operations without cluttering the main application logic.
